In [22]:
import pandas as pd
import re as regex

from collections import Counter
import csv
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np

In [3]:
def regex_partition(content, separator):
    separator_match = regex.search(separator, content)
    if not separator_match:
        return content, content, content

    matched_separator = separator_match.group(0)
    parts = regex.split(matched_separator, content, 1)

    return parts[0], matched_separator, parts[1]

def trim_domain_names(input_list):
    regexp = '_[0-9]'
    trimmed_domains = []
    for i in range(len(input_list)):
        if len(regex_partition(input_list[i], regexp)[0]) > 2:
            trimmed_domains.append(regex_partition(input_list[i], regexp)[0])
        else:
            trimmed_domains.append(input_list[i])
    regexp = '_C|_N'
    trimmed_domains_noCN = []
    for i in range(len(trimmed_domains)):
        trimmed_domains_noCN.append(regex_partition(trimmed_domains[i], regexp)[0])
        if len(regex_partition(trimmed_domains[i], regexp)[0]) > 2:
            trimmed_domains_noCN.append(regex_partition(trimmed_domains[i], regexp)[0])
        else:
            trimmed_domains_noCN.append(trimmed_domains[i])
    regexp = 'Peptidase'
    trimmed_domains_noCN_peptidasegrouped = []
    for i in range(len(trimmed_domains_noCN)):
        trimmed_domains_noCN_peptidasegrouped.append(regex_partition(trimmed_domains_noCN[i], regexp)[1])
    trimmed_domains_noCN_peptidasegrouped_cased = []
    for i in range(len(trimmed_domains_noCN_peptidasegrouped)):
        trimmed_domains_noCN_peptidasegrouped_cased.append(trimmed_domains_noCN_peptidasegrouped[i].lower())
    return trimmed_domains_noCN_peptidasegrouped_cased

In [4]:
genomes = [
'13FM-16-1',
'13FM-24-1',
'13FM-3-2',
'13FM-5-1',
'AG0004',
'AV1-1-1',
'BJ-90-611',
'BJ08-8',
'BN0123',
'BN0202',
'BN0252',
'CD0065',
'CD0142',
'CH0043',
'CH0072',
'CH0452',
'CH0461',
'CH1103',
'CH1164',
'CM0028',
'DB11-621',
'FJ0204804',
'FJ11SH-527-1',
'FJ12JN-084-3',
'FJ13SH05-2',
'FJ2000-62A',
'FJ2000-69A',
'FJ2001-112B',
'FJ2003-001A1',
'FJ2005113B',
'FJ2006-60A3',
'FJ72ZC7-77',
'FJ78-JJ',
'FJ81-JY',
'FJ81-MH',
'FJ81-SW',
'FJ81-ZP',
'FJ81221ZB11-14',
'FJ86-CT',
'FJ86061ZE3-39',
'FJ95054B',
'FJ95085AZB13-116',
'FJ98099',
'FJ99138',
'FJSH0703',
'GD06-53',
'GD08-2025',
'GY0040',
'HB-14',
'HB-LTH18',
'HN-0812-3',
'HN-158',
'HN10-1604',
'IA1',
'IB33',
'IB49',
'IC17',
'IE1K',
'IN0017',
'IN0054',
'IN0059',
'IN0114',
'IN0115',
'IN0116',
'IT0010',
'JS08-611',
'JS09-138',
'JX-09Z116-1',
'JX10-102',
'JX11-141',
'KA1-3-1',
'KA2-1-1',
'ML0060',
'ML0062',
'ML33',
'NG0012',
'NG0054',
'Nich-2-3-2',
'Nich-2-7-4',
'SC-10-120-65-2',
'SC-10-25-44-1',
'TG0004',
'TG0032',
'TN0001',
'TN0002',
'TN0050',
'TN0057',
'TN0090',
'TR0025',
'TW-1-1-1-B-1',
'TW-12CY-TB1-2',
'TW-12HL-DF1-2',
'TW-12HL-YL2-1',
'TW-12TD-RH1-1',
'TW-12TN-HB2-2',
'TW-12YL-DL3-2',
'TW-12YL-DP1-1',
'TW-12YL-TT4-1',
'TW-2-7-2-A-1',
'TW-6-2-2-B-1',
'TW-6-43-1',
'TW-CYBP1-3',
'TW-PT1-1',
'TW-PT3-1',
'TW-PT6-1',
'TW-TN4-2',
'US0041',
'VT0027',
'VT0030',
'WD-3-1',
'YN07205e',
'YN072310',
'YN072311',
'YN072313',
'YN08181e',
'YN08182c',
'YN126311',
'YN126441',
'YN8773-19',
'YN8773R-27',
'ZJ00-10',
'ZJ08-41',
'ZJ2011-7-1',
]

In [5]:
domain_counts_per_og = {}

for genome in genomes:
    kparse_out = "all_pfam_scan/"+ genome + ".pfamscan.kparse.out"
    df
    with open(kparse_out, newline = '') as file:
        file_reader = csv.reader(file, delimiter = '\t')
        for row in file_reader:
            gene = row[0]
            if gene in og_dict_no_70_15: # some genes dont have ogs
                og = og_dict_no_70_15[gene]
            if og not in domain_counts_per_og:
                domain_counts_per_og[og] = {}
            domains_trimmed = trim_domain_names(row[1].split("~"))
            domains = list(set(domains_trimmed))
            for domain in domains:
                if domain in domain_counts_per_og[og]:
                    domain_counts_per_og[og][domain] += 1
                else:
                    domain_counts_per_og[og][domain] = 1

NameError: name 'df' is not defined

In [19]:
df = pd.DataFrame()

for genome in genomes:
    df_temp = pd.read_csv("../pfam_enrichment/all_pfam_scan/"+ genome + ".pfamscan.kparse.out", sep='\t', header=None)
    df_temp.columns = ['id', 'annotation']
    df_temp['genome'] = df_temp['id'].str.split('_', expand=True)[2]
    df = pd.concat([df, df_temp], ignore_index=True, sort=False)

In [24]:
len(np.unique(df.genome))

123

In [27]:
pd.set_option('display.max_rows', 500)

In [34]:
df[((df['annotation'].str.contains('NACHT')) | (df['annotation'].str.contains('NB-ARC')))]

,id,annotation,genome
388,gene_5385_13FM-16-1_1_Baoshan,NACHT~Ank_2~Ank_2,13FM-16-1
545,gene_11524_13FM-16-1_1_Baoshan,NACHT,13FM-16-1
582,gene_10016_13FM-16-1_1_Baoshan,NB-ARC~TPR_12,13FM-16-1
843,gene_4457_13FM-16-1_1_Baoshan,NB-ARC~TPR_12~TPR_10,13FM-16-1
868,gene_4302_13FM-16-1_1_Baoshan,NB-ARC,13FM-16-1
...,...,...,...
915601,gene_1114_ZJ2011-7-1_2_NA,PNP_UDP_1~NACHT~Ank_2,ZJ2011-7-1
915604,gene_156_ZJ2011-7-1_2_NA,NACHT~Ank_2~Ank_2~Ank_2~Ank_2,ZJ2011-7-1
916299,gene_1929_ZJ2011-7-1_2_NA,RelA_SpoT~NACHT,ZJ2011-7-1
916877,gene_7210_ZJ2011-7-1_2_NA,NB-ARC,ZJ2011-7-1
